In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipaddress
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from datetime import datetime, timedelta
import math
import missingno as msno
plt.style.use('ggplot')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline
# sns.set(rc={'figure.figsize':(14,7)}) # For making the seaborn plots the same size

In [2]:
df = pd.read_csv('Cybersecurity_attacks.csv')

In [3]:
df['Attack category'].value_counts()

Exploits            69588
 Fuzzers            30145
DoS                 25117
Generic             20296
 Reconnaissance     18787
 Fuzzers             4216
Backdoor             4192
Analysis             1933
Reconnaissance       1783
 Shellcode           1314
Backdoors             259
Shellcode             227
Worms                 173
Reconnaissance          1
Name: Attack category, dtype: int64

In [4]:
df1 = df[(df['Attack category'] == 'Exploits') | (df['Attack category'] == 'Generic')]

In [5]:
df1.head()

,Attack category,Attack subcategory,Protocol,Source IP,Source Port,Destination IP,Destination Port,Attack Name,Attack Reference,.,Time
1,Exploits,Unix 'r' Service,udp,175.45.176.3,21223,149.171.126.18,32780,Solaris rwalld Format String Vulnerability (ht...,CVE 2002-0573 (http://cve.mitre.org/cgi-bin/cv...,.,1421927415-1421927415
2,Exploits,Browser,tcp,175.45.176.2,23357,149.171.126.16,80,Windows Metafile (WMF) SetAbortProc() Code Exe...,CVE 2005-4560 (http://cve.mitre.org/cgi-bin/cv...,.,1421927416-1421927416
3,Exploits,Miscellaneous Batch,tcp,175.45.176.2,13792,149.171.126.16,5555,HP Data Protector Backup (https://strikecenter...,CVE 2011-1729 (http://cve.mitre.org/cgi-bin/cv...,.,1421927417-1421927417
4,Exploits,Cisco IOS,tcp,175.45.176.2,26939,149.171.126.10,80,Cisco IOS HTTP Authentication Bypass Level 64 ...,CVE 2001-0537 (http://cve.mitre.org/cgi-bin/cv...,.,1421927418-1421927418
7,Generic,IXIA,tcp,175.45.176.0,29309,149.171.126.14,3000,Alt-N_MDaemon_WorldClient_Service_Memory_Corru...,CVE 2008-2631 (http://cve.mitre.org/cgi-bin/cv...,.,1421927420-1421927420


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89884 entries, 1 to 178029
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Attack category     89884 non-null  object
 1   Attack subcategory  89884 non-null  object
 2   Protocol            89884 non-null  object
 3   Source IP           89884 non-null  object
 4   Source Port         89884 non-null  int64 
 5   Destination IP      89884 non-null  object
 6   Destination Port    89884 non-null  int64 
 7   Attack Name         89884 non-null  object
 8   Attack Reference    89528 non-null  object
 9   .                   89884 non-null  object
 10  Time                89884 non-null  object
dtypes: int64(2), object(9)
memory usage: 8.2+ MB


In [7]:
df1 = df1.drop(columns=['Time', '.'])

In [8]:
def rem_junk(item):
    return item.strip(" ")

In [9]:
df1['Attack subcategory'] = df1['Attack subcategory'].apply(rem_junk)

In [10]:
df1['Destination Port'].value_counts()

 0        43542
 53       13917
 80       13563
 25        4936
 21        2262
          ...  
 45626        1
 50819        1
 27029        1
 12000        1
-146          1
Name: Destination Port, Length: 1715, dtype: int64

In [11]:
df2 = pd.get_dummies(df1, columns=[
    'Attack subcategory', 'Protocol', 'Source IP', 'Source Port',
    'Destination IP', 'Destination Port'])

In [12]:
df2 = df2.drop(columns=['Attack Name', 'Attack Reference'])

In [13]:
df2.head()

,Attack category,Attack subcategory_All,Attack subcategory_Apache,Attack subcategory_Backup Appliance,Attack subcategory_Browser,Attack subcategory_Browser FTP,Attack subcategory_Cisco IOS,Attack subcategory_Clientside,Attack subcategory_Clientside Microsoft,Attack subcategory_Clientside Microsoft Media Player,...,Destination Port_65455,Destination Port_65460,Destination Port_65470,Destination Port_65479,Destination Port_65486,Destination Port_65497,Destination Port_65502,Destination Port_65511,Destination Port_65517,Destination Port_65535
1,Exploits,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Exploits,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Exploits,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Exploits,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Generic,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def check_attack(item):
    if item == 'Exploits':
        return 1
    else:
        return 0

In [15]:
df2['Attack category'] = df2['Attack category'].apply(check_attack)

In [16]:
df3 = df2.drop(columns=['Attack category'])

In [18]:
data_train, data_test, labels_train, labels_test = train_test_split(df3, df2['Attack category'], test_size=0.30, random_state=110)

In [19]:
print(len(data_train),len(data_test),len(labels_train),len(labels_test))

62918 26966 62918 26966


In [20]:
random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(data_train,labels_train)

RandomForestClassifier()

In [21]:
# Predict the result for the test data
prediction_label = random_forest_classifier.predict(data_test)

In [22]:
cm = confusion_matrix(labels_test,prediction_label)
print(cm)
accuracy_score(labels_test,prediction_label)

[[ 6139    35]
 [   69 20723]]


0.9961432915523252